In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Install InstaGeo

In [ ]:
# Clone the InstaGeo-E2E-Geospatial-ML repository from GitHub
repository_url = "https://github.com/instadeepai/InstaGeo-E2E-Geospatial-ML"
!git clone {repository_url}

In [ ]:
%%bash
cd InstaGeo-E2E-Geospatial-ML
git pull

In [ ]:
%%capture
%%bash
# Navigate to the cloned InstaGeo-E2E-Geospatial-ML directory
cd /kaggle/working/InstaGeo-E2E-Geospatial-ML
# Stash any local changes to avoid conflicts when switching branches
git stash
#Switch to the 'geo-ai-hack' branch, which likely contains specific code for the Geo AI Hackathon
git checkout geo-ai-hack
# Install the InstaGeo package 
pip install -e .[all]

In [ ]:
# Import necessary libraries
import os
import re
import shutil
import yaml
import pandas as pd
import numpy as np
from pathlib import Path
from pyproj import CRS, Transformer
import rasterio
os.environ["HYDRA_FULL_ERROR"] ="1"

In [ ]:
def generate_label_mapping(root_dir, input_subdir, output_csv):
    """
    Generate a CSV mapping input chips to corresponding segmentation maps.

    Args:
        root_dir (str or Path): Root directory containing the subdirectories for chips and segmentation maps.
        input_subdir (str): Subdirectory path for chips within the root directory.
        output_csv (str or Path): Output path for the generated CSV file.
    """
    root_dir = Path(root_dir)
    chips_orig = os.listdir(root_dir / input_subdir / "chips")
    if os.path.exists(root_dir / input_subdir / "seg_maps"):
        add_label = True
    else:
        add_label = False

    chips = [chip.replace("chip", f"{input_subdir}/chips/chip") for chip in chips_orig]

    if add_label:
        seg_maps = [chip.replace("chip", f"{input_subdir}/seg_maps/seg_map") for chip in chips_orig]
        df = pd.DataFrame({"Input": chips, "Label": seg_maps})
    else:
        df = pd.DataFrame({"Input": chips})
    df.to_csv(output_csv, index=False)
    
    print(f"Number of rows is: {df.shape[0]}")
    print(f"CSV generated and saved to: {output_csv}")

In [ ]:
# set data folder path
input_dir="/kaggle/input/geo-ai-hack"

In [ ]:
# Generate label mappings for the training and testing datasets
generate_label_mapping(input_dir, 'hls_train/hls_train', "train_ds.csv")
generate_label_mapping(input_dir, 'hls_test/hls_test', "test_ds.csv")

### Validation Set

In [ ]:
def split_validation_data(mapping_csv, validation_split=0.3):
    """
    Split data into training and validation sets based on a CSV file mapping `chips` and `seg_maps`.

    Args:
        mapping_csv (str or Path): Path to the CSV file containing the mapping between `chips` and `seg_maps`.
        data_dir (str or Path): Path to the merged directory containing all files.
        validation_dir (str or Path): Path to the new directory for validation files.
        validation_split (float): Fraction of the data to use as the validation set.
    """
    df = pd.read_csv(mapping_csv)
    df = df.sample(frac=1, random_state=42).reset_index(drop=True)

    num_val = int(len(df) * validation_split)
    train_df = df[num_val:]
    val_df = df[:num_val]
    train_df.to_csv("train_split.csv",index=False)    
    print(f"CSV train split  saved to: train_split.csv")
    val_df.to_csv("validation_split.csv",index=False)    
    print(f"CSV validation split  saved to: validation_split.csv")
    
    return 
    

In [ ]:
# Split the training dataset into training and validation sets
split_validation_data(
    mapping_csv="train_ds.csv",
    validation_split=0.3
)

## InstaGeo - Model

After creating our training and validation splits, we can move on to fine-tuning a model that includes a Prithvi backbone paired with a classification head. For regression tasks, the classification head can easily be replaced with a suitable regression head. Additionally, if a completely different model architecture is needed, it can be designed and implemented within this framework.

In [ ]:
def load_yml(filepath):
    """Load data from a YAML file.

    Args:
        filepath (str | Path): The path to the YAML file.

    Returns:
        Dict: The loaded data, or None if the file does not exist.
    """
    filepath=Path(filepath)
    with filepath.open() as f:
        return yaml.safe_load(f)
        
def save_yml(data,filepath):
    """Save data to a YAML file.

    Args:
        data (Dict): The data to save.
        filepath (str | Path): The file path to save the YAML to.
    """
    filepath = Path(filepath)
    with filepath.open("w") as f:
        yaml.dump(data, f)
    print(f"Data saved to {filepath}.")


### Launch Training


First, compute the mean and standard deviation for the dataset using the InstaGeo command. Then update the corresponding configuration file [locust.yaml](https://github.com/instadeepai/InstaGeo-E2E-Geospatial-ML/blob/main/instageo/model/configs/locust.yaml). In this case, it has already been done for you. However, if you change the dataset split or modify the training data, you should run the command again to compute the new mean and standard deviation.

In [ ]:
# %%bash
# python -m instageo.model.run --config-name=locust \
#     root_dir="/kaggle/input/geo-ai-hack" \
#     train.batch_size=8 \
#     train.num_epochs=5 \
#     mode=stats \
#     train_filepath="train_ds.csv" \

In [ ]:
# Updat locust file
# Load the Locust model configuration file
locust_cfg_path="InstaGeo-E2E-Geospatial-ML/instageo/model/configs/locust.yaml"
# Load the YAML configuration into a dictionary
locust_cfg=load_yml(locust_cfg_path)
# Update the mean and standard deviation values in the configuration
locust_cfg["mean"]=[670.5441284179688, 1267.7974853515625, 1772.599365234375, 2415.69091796875, 2879.2431640625, 2337.822509765625]
locust_cfg["std"]=[2146.305419921875, 2203.416259765625, 2247.03515625, 2310.74755859375, 2322.708984375, 2211.968505859375]
# Save the updated configuration back to the YAML file
save_yml(locust_cfg,locust_cfg_path)

In [ ]:
# Train the InstaGeo model using the Locust configuration
!python -m instageo.model.run  --config-name=locust \
    hydra.run.dir="/kaggle/working/outputs/first_run" \
    root_dir="/kaggle/input/geo-ai-hack" \
    train.batch_size=8 \
    train.num_epochs=5 \
    mode=train \
    train_filepath="train_split.csv" \
    valid_filepath="validation_split.csv"

### Run Model Evaluation
 To evaluate the model, adjust the `checkpoint_path` argument to point to the desired model checkpoint. The checkpoint file is typically located in the `hydra.run.dir` directory and is named `instageo_best_checkpoint.ckpt`.
For example:
```
/kaggle/working/outputs/first_run/instageo_best_checkpoint.ckpt 
```
Make sure to provide the correct path to the checkpoint file based on your training output directory.

In [ ]:
%%bash
python -m instageo.model.run --config-name=locust \
    root_dir="/kaggle/input/geo-ai-hack" \
    test_filepath="validation_split.csv" \
    train.batch_size=16 \
    checkpoint_path='/kaggle/working/outputs/first_run/instageo_best_checkpoint.ckpt' \
    mode=eval

## Make Submission

We first run inference on test chips to get the predictions

In [ ]:
%%bash
python -m instageo.model.run --config-name=locust \
    root_dir="/kaggle/input/geo-ai-hack" \
    test_filepath="test_ds.csv" \
    train.batch_size=16 \
    checkpoint_path='/kaggle/working/outputs/first_run/instageo_best_checkpoint.ckpt' \
    output_dir='/kaggle/working/predictions' \
    mode=chip_inference

After getting the prdictions for each chip, we retrieve the predicted value for each observatio in our test split.

In [ ]:


predictions_directory = "/kaggle/working/predictions"
prediction_files = os.listdir(predictions_directory)

def get_prediction_value(row):
    matching_files = [f for f in prediction_files if (str(row['date']) in f) and (row['mgrs_tile_id'] in f)]
    if not matching_files:
        return (np.nan, np.nan)
    for file in matching_files:
        with rasterio.open(f"{predictions_directory}/{file}") as src:
            width, height = src.width, src.height
            affine_transform = rasterio.transform.AffineTransformer(src.transform)
            transformer = Transformer.from_crs(CRS.from_epsg(4326), src.crs, always_xy=True)
            x_chip, y_chip = transformer.transform(row['x'], row['y'])
            x_offset, y_offset = affine_transform.rowcol(x_chip, y_chip)
            
            if 0 <= x_offset < width and 0 <= y_offset < height:
                return src.read(1)[y_offset, x_offset], file
    return (np.nan, np.nan)

In [ ]:
submission_df = pd.read_csv("/kaggle/input/geo-ai-hack/test.csv")

submission_df[['prediction', 'filename']] = submission_df.apply(get_prediction_value, axis=1, result_type='expand')
submission_df[["id","prediction"]].to_csv("hls_submission.csv",index=False)